In [1]:
from numpy.random.mtrand import randint
import torch
from torch import nn
import numpy as np
import gzip
import json
import os
import urllib
import Goban
import torch.optim as optim
from Goban import Board

In [2]:
def get_raw_data_go():
    ''' Returns the set of samples from the local file or download it if it does not exists'''

    raw_samples_file = "samples-9x9.json.gz"

    if not os.path.isfile(raw_samples_file):
        print("File", raw_samples_file, "not found, I am downloading it...", end="")
        urllib.request.urlretrieve("https://www.labri.fr/perso/lsimon/ia-inge2/samples-9x9.json.gz", "samples-9x9.json.gz")
        print(" Done")

    with gzip.open("samples-9x9.json.gz") as fz:
        data = json.loads(fz.read().decode("utf-8"))
    return data

In [3]:
class ConvolutionBlock(nn.Module):
    def __init__(self):
        super(ConvolutionBlock, self).__init__()
        self.conv = nn.Conv2d(9, 9,kernel_size=5, padding='same')
        self.bn = nn.BatchNorm2d(9)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        conv = self.conv(x)
        conv += x
        x1 = self.bn(conv)
        return self.relu(x1)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.res1 = ConvolutionBlock()
        self.res2 = ConvolutionBlock()
        self.res3 = ConvolutionBlock()
        self.res4 = ConvolutionBlock()
        self.res5 = ConvolutionBlock()
        self.linear1 = nn.Linear(9 * 81 * 2, 82)
        self.linear2 = nn.Linear(9 * 81 * 2, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(0)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)

    def forward(self, x):
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        #x = self.res4(x)
        #x = self.res5(x)
        x = torch.flatten(x)
        p = self.softmax(self.dropout1(self.linear1(x.clone())))
        v = self.sigmoid(self.dropout2(self.linear2(x.clone())))
        
        return (p,v)


In [4]:
def name_to_coord(s):
    assert s != "PASS"
    indexLetters = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'J':8}

    col = indexLetters[s[0]]
    lin = int(s[1:]) - 1
    return col, lin

def rotation(board):
    L = []
    L.append(np.reshape(board, 81))
    for i in range(1, 4):
        rotation = np.rot90(board, i)
        L.append(np.reshape(rotation, 81))
    return L

def symmetry(board):
    L = rotation(np.reshape(board, (9,9))) + rotation(np.transpose(np.reshape(board, (9,9))))
    return L

def coord_to_board(black_moves, white_moves):
    board = [[0. for i in range(9)] for j in range(9)]
    for x,y in black_moves:
        board[x][y] = 1.
    for x,y in white_moves:
        board[x][y] = -1.
    return board

In [21]:
def get_data(data):
    D = []
    j = 0
    length = len(data)
    for game in data:
        print(j, "/", length)
        j += 1
        if j == 50:
            break
        if game['depth'] < 9:
            continue
        board_black_list = []
        board_white_list = []
        board = Goban.Board()
        err = False
        for m in game['list_of_moves']:
            if m == 'PASS':
                move = -1
            else:
                (x,y) = name_to_coord(m)
                move = y * board._BOARDSIZE + x
            try:
                board.push(move)
            except:
                err = True
                break
            
            board_black = board._board.copy()
            board_white = board._board.copy()
            for i in range(board._BOARDSIZE ** 2):
                if board_black[i] == 2:
                    board_black[i] = 0
                if board_white[i] == 1:
                    board_white[i] = 0
                if board_white[i] == 2:
                    board_white[i] = 1 
            board_black_list.append(np.array(board_black))
            board_white_list.append(np.array(board_white))
        if err:
            continue
        hist_black = board_black_list[-9:-1]
        hist_white = board_black_list[-9:-1]
        hist_black_s = [symmetry(board) for board in hist_black]
        hist_white_s = [symmetry(board) for board in hist_white]
        for sym in range(8):
            hist = [np.array([hist_black_s[i][sym], hist_white_s[i][sym]]) for i in range(8)]
            v = None
            if (game['depth'] - 1) %2 == 0:
                hist.append(np.array([np.ones(board._BOARDSIZE ** 2, dtype=np.uint8), np.ones(board._BOARDSIZE ** 2, dtype=np.uint8)])) #BLACK to 1
                v = int(game['black_wins']) / 100
            else:
                hist.append(np.array([np.zeros(board._BOARDSIZE ** 2, dtype=np.uint8), np.zeros(board._BOARDSIZE ** 2, dtype=np.uint8)])) #WHITE to 0
                v = int(game['white_wins']) / 100
            hist = np.array(hist) 
            m = game['list_of_moves'][-1]
            if m == 'PASS':
                move = board._BOARDSIZE ** 2
            else:
                (x,y) = name_to_coord(m)
                move = y * board._BOARDSIZE + x
            p = np.zeros(board._BOARDSIZE ** 2 + 1, dtype=np.uint8)
            p[move] = 1

            d = torch.reshape(torch.from_numpy(hist), (1,9,2,81)).type(torch.float32)
            D.append((d, torch.tensor(p, dtype=torch.float32), torch.tensor(v, dtype=torch.float32)))
    return D

In [22]:
data = get_raw_data_go()
D = get_data(data)

0 / 41563
1 / 41563
2 / 41563
3 / 41563
4 / 41563
5 / 41563
6 / 41563
7 / 41563
8 / 41563
9 / 41563
10 / 41563
11 / 41563
12 / 41563
13 / 41563
14 / 41563
15 / 41563
16 / 41563
17 / 41563
18 / 41563
19 / 41563
20 / 41563
21 / 41563
22 / 41563
23 / 41563
24 / 41563
25 / 41563
26 / 41563
27 / 41563
28 / 41563
29 / 41563
30 / 41563
31 / 41563
32 / 41563
33 / 41563
34 / 41563
35 / 41563
36 / 41563
37 / 41563
38 / 41563
39 / 41563
40 / 41563
41 / 41563
42 / 41563
43 / 41563
44 / 41563
45 / 41563
46 / 41563
47 / 41563
48 / 41563
49 / 41563


In [27]:
model = NeuralNetwork()
bacth_size = 64
def train():
    criterion = nn.L1Loss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
    #optimizer = optim.Adam(model.parameters(), lr=0.001)
    length = len(D)
    for step in range(100):
        i = 0
        step_loss = None
        while i < length: 
            optimizer.zero_grad()
            total_loss = None
            for b in range(bacth_size):
                if i + b == length:
                    break
                (d, p, v) = D[i + b]
                p_model, v_model = model(d.clone().detach().requires_grad_(True))
                loss = criterion(p_model, p)
                loss /= (Board._BOARDSIZE ** 2 + 1)
                loss += torch.abs(v - v_model[0])
                if total_loss is None:
                    total_loss = loss
                else:
                    total_loss += loss
            total_loss /= b
            if step_loss is None:
                step_loss = total_loss
            else:
                step_loss += total_loss
                
            #print("loss:", float(total_loss))
            total_loss.backward(retain_graph=True)
            optimizer.step()
            i += bacth_size
        print("Step", step,":", float(step_loss))

In [28]:
torch.autograd.set_detect_anomaly(True)
train()

Step 0 : 16.449241638183594
Step 1 : 15.584203720092773
Step 2 : 15.124652862548828
Step 3 : 14.615053176879883
Step 4 : 14.310062408447266
Step 5 : 13.840622901916504
Step 6 : 13.146416664123535
Step 7 : 13.197901725769043
Step 8 : 12.604692459106445
Step 9 : 12.742905616760254
Step 10 : 12.31273365020752
Step 11 : 12.023510932922363
Step 12 : 11.804462432861328
Step 13 : 11.85515022277832
Step 14 : 11.240861892700195
Step 15 : 11.564504623413086
Step 16 : 11.707124710083008
Step 17 : 11.489660263061523
Step 18 : 11.07580280303955
Step 19 : 10.687670707702637
Step 20 : 11.475811004638672
Step 21 : 10.632884979248047
Step 22 : 10.072452545166016
Step 23 : 10.333772659301758
Step 24 : 10.45167064666748
Step 25 : 9.569406509399414
Step 26 : 10.05273723602295
Step 27 : 10.433273315429688
Step 28 : 9.708049774169922
Step 29 : 9.699769020080566
Step 30 : 9.611945152282715
Step 31 : 9.33121109008789
Step 32 : 9.498335838317871
Step 33 : 9.473689079284668
Step 34 : 9.407791137695312
Step 35 :